In [1]:
%pip install llama-index
%pip install llama-index-llms
%pip install llama-index-readers
%pip install llama-index-embeddings
%pip install dotenv

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement llama-index-llms (from versions: none)
ERROR: No matching distribution found for llama-index-llms
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement llama-index-readers (from versions: no

## Load Credentials

In [5]:
from __future__ import print_function
import logging
import sys
import os
import pandas as pd
import glob
from dotenv import load_dotenv
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import ( Settings, VectorStoreIndex, SimpleDirectoryReader)
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
import chromadb
from llama_index.core.callbacks import CallbackManager
import datetime
import numpy as np
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter
from sqlalchemy import *
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.node_parser import SentenceWindowNodeParser

logging.getLogger().setLevel(logging.WARNING)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

load_dotenv('../Credentials/.env')

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
credential = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_api_version = "2024-04-01-preview"
azure_openai_embedding_deployment = "text-embedding-ada-002"
embedding_model_name = "text-embedding-ada-002"
llm_model_name = "gpt-35-turbo-16k"
api_type = "azure"

## Load ChromaDB

In [6]:
remote_db = chromadb.HttpClient(host='localhost',port=8000)
remote_db.delete_collection("quickstart")
chroma_collection = remote_db.get_or_create_collection("quickstart")

## Load Data, Get Metadata

In [7]:
reader = SimpleDirectoryReader("../Data/", recursive=True, filename_as_id=True, required_exts=[".pdf", ".docx", ".xlsx", ".pptx"])

pd.set_option('future.no_silent_downcasting', True)

documents = []
for docs in reader.iter_data():
    for filename in glob.glob('../Data/course*/course_file.json',recursive=True):
        df = pd.read_json(filename)

        file_metadata = df.loc[df['filename'] == docs[0].metadata['file_name'].replace('_','+')]
        if file_metadata.empty != True:                
            file_metadata = file_metadata.squeeze().to_dict()
            file_metadata = pd.DataFrame(file_metadata, index=[0]).replace(np.NaN, 0).replace(0, None)
            file_metadata = file_metadata.to_dict('records')[0]
            folder = file_metadata.get('folder_id')
        else:
            file_metadata = {}
            folder = ''
            pass

    for filename in glob.glob('../Data/course*/course_folder.json',recursive=True):
        df = pd.read_json(filename)

        folder_metadata = df.loc[df['id'] == folder]
        if folder_metadata.empty != True:
            folder_metadata = folder_metadata.squeeze().to_dict()
            folder_metadata = pd.DataFrame(folder_metadata, index=[0]).replace(np.NaN, 0).replace(0, None)
            folder_metadata = folder_metadata.to_dict('records')[0]
            if 'Week' in folder_metadata['full_name']:
                week = [i for i in folder_metadata['full_name'].split("/") if 'Week' in i][0].replace('Week','').replace(' ','')
                folder_metadata.update({"week":week})
        else:
            folder_metadata = {}
            pass
    
    for filename in glob.glob('../Data/course*/course_course.json',recursive=True):
        df = pd.read_json(filename)        
        course_metadata = df.loc[df['id'] == folder_metadata.get('context_id')]
        if course_metadata.empty != True:
            course_metadata = course_metadata.squeeze().to_dict()
            course_id = folder_metadata.get('context_id')
        else:
            course_metadata = {}
            pass
        
    for doc in docs:
        doc.metadata.update({"file_id": file_metadata.get('id'), "folder_id":file_metadata.get('folder_id'), "display_name":file_metadata.get('display_name')})
        doc.metadata.update({"week": folder_metadata.get('week')})
        doc.metadata.update({"course_id": course_metadata.get('id'), "course_name":course_metadata.get('name'),"course_code":course_metadata.get('course_code'),"course_term":course_metadata.get('term', {}).get('name')}) 

    documents.extend(docs)


In [4]:
documents[0].metadata

{'page_label': '1',
 'file_name': '1-s2.0-S1538544221000821-main.pdf',
 'file_path': '../Data/course113113/downloads/1-s2.0-S1538544221000821-main.pdf',
 'file_type': 'application/pdf',
 'file_size': 719173,
 'creation_date': '2024-04-16',
 'last_modified_date': '2024-04-16',
 'last_accessed_date': '2024-04-30',
 'file_id': 19434347,
 'folder_id': 3847349,
 'display_name': '1-s2.0-S1538544221000821-main.pdf',
 'week': '4',
 'course_id': 131972,
 'course_name': 'PWY 133: Integrated Human Pathophysiology II - HMS - 01/29/2024 - 03/15/2024',
 'course_code': 'PWY 133',
 'course_term': '2023-2024 Spring'}

## Node Parsing, Index Creation

In [8]:
# set up ChromaVectorStore and load in data
llm = AzureOpenAI(
            model = llm_model_name,
            deployment_name = llm_model_name,
            api_key = credential,
            azure_endpoint = endpoint,
            api_version = azure_openai_api_version,
            api_type = api_type
        )

embed_model = AzureOpenAIEmbedding(
            model = embedding_model_name,
            deployment_name = embedding_model_name,
            api_key = credential,
            azure_endpoint = endpoint,
            api_version = azure_openai_api_version,
            api_type = api_type,
            embed_batch_size=50
        )

Settings.llm = llm
Settings.embed_model = embed_model

# create the sentence window node parser w/ default settings
sentence_node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text"
)

nodes = sentence_node_parser.get_nodes_from_documents(documents)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection, nodes=nodes)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)


**********
Trace: index_construction
    |_node_parsing ->  1.666124 seconds
      |_chunking ->  0.004408 seconds
      |_chunking ->  0.001325 seconds
      |_chunking ->  0.004937 seconds
      |_chunking ->  0.0057 seconds
      |_chunking ->  0.004852 seconds
      |_chunking ->  0.005128 seconds
      |_chunking ->  0.00575 seconds
      |_chunking ->  0.006164 seconds
      |_chunking ->  0.004544 seconds
      |_chunking ->  0.00546 seconds
      |_chunking ->  0.013612 seconds
      |_chunking ->  0.012187 seconds
      |_chunking ->  0.010459 seconds
      |_chunking ->  0.006179 seconds
      |_chunking ->  0.008066 seconds
      |_chunking ->  0.009649 seconds
      |_chunking ->  0.001593 seconds
      |_chunking ->  0.006976 seconds
      |_chunking ->  0.006188 seconds
      |_chunking ->  0.008466 seconds
      |_chunking ->  0.008975 seconds
      |_chunking ->  0.012439 seconds
      |_chunking ->  0.022024 seconds
      |_chunking ->  0.00123 seconds
      |_chunking

## Save to Persistent Storage

In case you want to load your index later, saving you from having to re-parse your documents every time

In [ ]:
db = chromadb.PersistentClient(path="../chroma_db")

## Create Query Engine, Ask a Question

In [9]:
# filters = MetadataFilters(
#     filters=[MetadataFilter(key="course_id", value=131972)]
# )

filters = MetadataFilters(filters=[
    MetadataFilter(
        key="course_id", 
        value=131972
    ),
     MetadataFilter(
        key="week", 
        value='2'
    ),
])

# query_engine = index.as_query_engine(filters=filters)
# response = query_engine.query("Give me a multiple choice question from the context")
# print(response)

chat_engine = index.as_chat_engine(filters=filters)
streaming_response = chat_engine.stream_chat("What is the material in the context?")
for token in streaming_response.response_gen:
    print(token, end="")

**********
Trace: chat
    |_agent_step ->  0.77021 seconds
      |_llm ->  0.0 seconds
**********
I'm sorry, but I don't have any context to determine what material you are referring to. Could you please provide more information or specify the context?